In [1]:
%%capture

# get_corpus_path
# get_txt_orig_path
# get_txt_clean_path
%run ../path_manager.ipynb


# CorpusCleaner
%run ../DataCleanerModule.ipynb

## Jupyter.notebook.save_checkpoint()

In [4]:
ENCHANT_INSTALLED

True

# Misspelling correction

In [5]:
CORPUS_ID = 'WB'
TXT_ORIG_DIR = get_txt_orig_path(CORPUS_ID)
# TXT_CLEAN_DIR = os.path.join(get_corpus_path(CORPUS_ID), 'TXT_PCLEAN')  # 
TXT_CLEAN_DIR = get_txt_clean_pos_path(CORPUS_ID)

metadata_file = pd.read_csv(os.path.join(get_corpus_path(CORPUS_ID), f'{CORPUS_ID.lower()}_metadata.csv'), low_memory=False)
metadata_file = metadata_file.dropna(axis=0, subset=['url_txt'])

print(metadata_file.shape)
metadata_file.head(5)

In [10]:
# %%time
# cleaner=ParallelCorpusCleaner(
#     use_spellchecker=True, use_respeller=True, use_lemmatizer=True, use_spacy=True,
#     replacements_plurals_to_singular_file='../whitelists/whitelist_replacements_plurals_to_singular.csv',
#     acronyms_file='../whitelists/whitelist_acronyms.csv',
#     extract_only_pos=('NOUN', 'ADJ'),
# )

# cleaner.set_input_folder(TXT_ORIG_DIR)
# cleaner.set_output_folder(TXT_CLEAN_DIR)

# Run with subset dataset

In [13]:
os.path.join(TXT_CLEAN_DIR, 'wb_19431317.txt')

'/home/wb536061/wb-nlp/CORPUS/WB/TXT_CLEAN_POS/wb_19431317.txt'

In [16]:
import pickle
import joblib
from joblib import Parallel, delayed, parallel_backend, wrap_non_picklable_objects

# os.environ['LOKY_PICKLER'] = 'pickle'
joblib.__version__

In [17]:
process_output_manager = None  # multiprocessing.Manager()
unprocessed_shuffled_files = doclist = [f"{id}.txt" for id in metadata_file['id'].sample(n=len(metadata_file), random_state=1029)]

In [20]:
n_jobs = 75
num_docs = 500  # 500
notify_fname = 'pos_tag_cleaner.log'
clean_doc_outputs = []

cleaner=ParallelCorpusCleaner(
    use_spellchecker=True, use_respeller=True, use_lemmatizer=True, use_spacy=True,
    replacements_plurals_to_singular_file='../whitelists/whitelist_replacements_plurals_to_singular.csv',
    acronyms_file='../whitelists/whitelist_acronyms.csv',
    extract_only_pos=('NOUN', 'ADJ'),
)


def clean_docs_wrapper(fileids, skip_existing=True):


    cleaner.set_input_folder(TXT_ORIG_DIR)
    cleaner.set_output_folder(TXT_CLEAN_DIR)

    payloads = []

    for fileid in fileids:

        filen = os.path.join(cleaner.input_folder, fileid)
        newfile = os.path.join(cleaner.output_folder, fileid)

        if os.path.isfile(newfile) and skip_existing:
            payload = cleaner.load_existing_and_extract_metadata(fileid, newfile, save_docs=True, process_output_dict=None)
        else:
            payload = cleaner.clean_doc(fileid, filen, save_doc=True, process_output_dict=None)

        payloads.append(payload)

    return payloads


print(n_jobs)

while True:
    
    try:        
        with Parallel(n_jobs=n_jobs, batch_size='auto', pre_dispatch=2*n_jobs, backend='loky') as parallel:
            # batch_size of 10 means that the process will receive 10 list of fnames.
            # pre_dispatch of 70 means that joblib will send 70 batches to the queue.
            batch = []
            batch_count = 0
            for fnames in np.array_split(unprocessed_shuffled_files, len(unprocessed_shuffled_files) // num_docs + 1):
                if len(batch) and len(batch) % (n_jobs * 2) == 0:
                    batch_count += 1
                    parsed_datasets = parallel(delayed(clean_docs_wrapper)(fnames, True) for fnames in batch)
                    batch = []
                    parsed_datasets = [payload for parsed_dataset in parsed_datasets for payload in parsed_dataset]
                    clean_doc_outputs.extend(parsed_datasets)
                    notify(notify_fname, f'Processed {len(parsed_datasets)} docs for batch {batch_count} with total of {len(clean_doc_outputs)} docs...', verbose=True)
                    # store_batch(parsed_datasets, batch_count)
                    # insert_result = collection.insert_many(parsed_datasets)
                    notify(notify_fname, f'Stored {len(parsed_datasets)} docs for batch {batch_count} with total of {len(clean_doc_outputs)} docs...', verbose=True)
                    # notify(notify_fname, f'Stored {len(insert_result.inserted_ids)} docs for batch {batch_count}...', verbose=True)
                else:
                    batch.append(fnames)

            if batch:
                batch_count += 1
                parsed_datasets = parallel(delayed(clean_docs_wrapper)(fnames, True) for fnames in batch)
                batch = []
                parsed_datasets = [payload for parsed_dataset in parsed_datasets for payload in parsed_dataset]
                clean_doc_outputs.extend(parsed_datasets)
                notify(notify_fname, f'Processed {len(parsed_datasets)} docs for batch {batch_count} with total of {len(clean_doc_outputs)} docs...', verbose=True)
                # store_batch(parsed_datasets, batch_count)
                # insert_result = collection.insert_many(parsed_datasets)
                notify(notify_fname, f'Stored {len(parsed_datasets)} docs for batch {batch_count} with total of {len(clean_doc_outputs)} docs...', verbose=True)
                # notify(notify_fname, f'Stored {len(insert_result.inserted_ids)} docs for batch {batch_count}...', verbose=True)
        break
        
    except pickle.PicklingError:
        notiexcepttify_fname, 'Retrying...', verbose=True)
        continue
        
    except Exception as e:
        excep = e
        raise(e)
        break

if len(clean_doc_outputs):
    output = {k: {} for k in clean_doc_outputs[0]}

    for o in clean_doc_outputs:
        for k in output:
            output[k].update(o[k])

Doclen 18 < 50 = None
75
2020-03-05 01:35:42.256092: Processed 75000 docs for batch 1 with total of 75000 docs...
2020-03-05 01:35:42.256593: Stored 75000 docs for batch 1 with total of 75000 docs...


/home/wb536061/anaconda3/envs/wb-nlp/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


2020-03-05 03:02:05.080444: Processed 74932 docs for batch 2 with total of 149932 docs...
2020-03-05 03:02:05.080798: Stored 74932 docs for batch 2 with total of 149932 docs...
2020-03-05 04:29:57.378423: Processed 74850 docs for batch 3 with total of 224782 docs...
2020-03-05 04:29:57.378754: Stored 74850 docs for batch 3 with total of 224782 docs...
2020-03-05 05:00:33.096132: Processed 19960 docs for batch 4 with total of 244742 docs...
2020-03-05 05:00:33.096625: Stored 19960 docs for batch 4 with total of 244742 docs...


In [22]:
batch_count, len(unprocessed_shuffled_files), len(output)

(4, 246240, 244742)

In [72]:
output = {k: {} for k in clean_doc_outputs[0]}

for o in clean_doc_outputs:
    for k in output:
        output[k].update(o[k])

In [23]:
# %%time
# # # doclist=['436966.txt', '2002839.txt']

# doclist = [f"{id}.txt" for id in metadata_file['id'].sample(n=len(metadata_file), random_state=1029)]

# print(len(doclist))
# # wb_8886341
# print(pd.datetime.now())
# output=cleaner.batch_clean_docs(
#     doclist,
# #     default_docs_per_worker=10,
#     batch_size=96,  # x4 of num_workers
#     save_docs=True,
#     skip_existing=True,
#     collect_spell_errors=False,
#     process_output_manager=process_output_manager
# )
# print(pd.datetime.now())

In [24]:
# 246240
# 2020-03-03 21:47:20.486896
# Doclen 7 < 50 = wb_31422591
# Doclen 5 < 50 = wb_24998598
# Doclen 40 < 50 = wb_18225021
# Exception received: 11
# Empty doc post lemmatizer = wb_749898

### Convert the output into a dataframe for easier processing

In [73]:
misc_metadata = pd.DataFrame(output)
misc_metadata.shape

(243178, 7)

In [75]:
misc_metadata.to_csv(os.path.join(get_corpus_path(CORPUS_ID), f'{CORPUS_ID.lower()}_pos_derived_metadata.csv'))

In [76]:
misc_metadata.head(2)

,lang,tokens,text,spell_errors,skipped,exception,write_status
wb_6448332,"(en, 0.8571408354492007)",0,page short term consultant project environment...,[],Not in english | en:0.8571408354492007,None,False
wb_31189916,"(en, 0.9999952109019343)",717,implementation status result climate resilienc...,"[subproject, sluicegate, monre, mgmt, undisbur...",,None,True


In [16]:
misc_metadata.head(2)

,lang,tokens,text,spell_errors,skipped,exception,write_status
wb_31422591,"(ERROR, 0)",0,NaN,NaN,Doclen 7 < 50 = wb_31422591,None,False
wb_19200456,"(en, 0.9999953724491716)",385,NaN,NaN,,None,True


Log items that were skipped due to some unexpected errors

In [77]:
misc_metadata[misc_metadata.skipped != ''].skipped.to_csv(f'{CORPUS_ID.lower()}_pos_skipped-items-{pd.datetime.now()}.log')

/home/wb536061/anaconda3/envs/wb-nlp/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


Generate new attributes based on the `lang` column from `misc_metadata`.

In [78]:
misc_metadata['language_detected'] = misc_metadata.lang.map(lambda x: x[0])
misc_metadata['language_score'] = misc_metadata.lang.map(lambda x: x[1])
misc_metadata = misc_metadata.drop('lang', axis=1)

misc_metadata.index.name = 'id'
misc_metadata = misc_metadata.reset_index()

In [79]:
misc_metadata.head(2)

,id,tokens,text,spell_errors,skipped,exception,write_status,language_detected,language_score
0,wb_6448332,0,page short term consultant project environment...,[],Not in english | en:0.8571408354492007,None,False,en,0.857141
1,wb_31189916,717,implementation status result climate resilienc...,"[subproject, sluicegate, monre, mgmt, undisbur...",,None,True,en,0.999995


In [91]:
detect_langs(misc_metadata.loc[0]['text'])

[en:0.9999967186647158]

In [19]:
misc_metadata.head(2)

,id,tokens,text,spell_errors,skipped,exception,write_status,language_detected,language_score
0,wb_31422591,0,NaN,NaN,Doclen 7 < 50 = wb_31422591,None,False,ERROR,0.000000
1,wb_19200456,385,NaN,NaN,,None,True,en,0.999995


In [92]:
misc_metadata[misc_metadata.skipped != ''].head()

,id,tokens,text,spell_errors,skipped,exception,write_status,language_detected,language_score
0,wb_6448332,0,page short term consultant project environment...,[],Not in english | en:0.8571408354492007,None,False,en,0.857141
13,wb_31422591,0,﻿,[],Doclen 7 < 50 = wb_31422591,None,False,ERROR,0.000000
25,wb_30551697,0,procurement plan text font instruction staff f...,[],Not in english | en:0.7142821602262297,None,False,en,0.714282
32,wb_9917358,0,page copy number agreement page number financi...,[],Not in english | en:0.8571380696762061,None,False,en,0.857138
36,wb_29562458,0,original problem text extraction pdftotext una...,[],Not in english | ca:0.7142810488487307,None,False,ca,0.714281


In [20]:
misc_metadata[misc_metadata.skipped != ''].head()

,id,tokens,text,spell_errors,skipped,exception,write_status,language_detected,language_score
0,wb_31422591,0,NaN,NaN,Doclen 7 < 50 = wb_31422591,None,False,ERROR,0.000000
6,wb_24998598,0,NaN,NaN,Doclen 5 < 50 = wb_24998598,None,False,ERROR,0.000000
27,wb_18225021,0,NaN,NaN,Doclen 40 < 50 = wb_18225021,None,False,ERROR,0.000000
61,wb_749898,0,NaN,NaN,Empty doc post lemmatizer = wb_749898,None,False,ERROR,0.000000
72,wb_29562458,0,NaN,NaN,Not in english | ca:0.9999942797661113,None,False,ca,0.999994


In [93]:
misc_metadata['id'].nunique(), misc_metadata.shape

(243178, (243178, 9))

In [21]:
misc_metadata['id'].nunique(), misc_metadata.shape

(244661, (244661, 9))

In [94]:
valid_processed_ids = misc_metadata[(misc_metadata.skipped == '') & (misc_metadata.write_status == True)].set_index('id').index
len(valid_processed_ids)

206897

In [22]:
valid_processed_ids = misc_metadata[(misc_metadata.skipped == '') & (misc_metadata.write_status == True)].set_index('id').index
len(valid_processed_ids)

217949

In [95]:
complete_metadata_file = pd.concat([metadata_file.drop_duplicates('id').set_index('id'), misc_metadata.set_index('id')[['language_detected', 'language_score', 'tokens', 'write_status']]], axis=1)
complete_metadata_file = complete_metadata_file.loc[valid_processed_ids]

/home/wb536061/anaconda3/envs/wb-nlp/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [96]:
complete_metadata_file.shape

(206897, 32)

In [97]:
complete_metadata_file.head(2)

,corpus,path_original,path_clean,filename_original,year,major_doc_type,doc_type,author,collection,title,...,wb_product_line,wb_major_theme,wb_theme,wb_sector,wb_subtopic_src,wb_project_id,language_detected,language_score,tokens,write_status
id,,,,,,,,,,,,,,,,,,,,,
wb_31189916,wb,/NLP/CORPUS/WB/TXT_ORIG/wb_31189916.txt,/NLP/CORPUS/WB/TXT_CLEAN/wb_31189916.txt,Disclosable-Version-of-the-ISR-Mekong-Delta-In...,2019.0,Project Documents,Implementation Status and Results Report,NaN,NaN,Disclosable Version of the ISR - Mekong Delta ...,...,NaN,NaN,NaN,NaN,"Climate Change Impacts,Labor Markets,Global En...",P153544,en,0.999995,717.0,True
wb_17220866,wb,/NLP/CORPUS/WB/TXT_ORIG/wb_17220866.txt,/NLP/CORPUS/WB/TXT_CLEAN/wb_17220866.txt,E21050v400HA0N0IC0disclosed02050130.txt,2012.0,Project Documents,Environmental Assessment,NaN,NaN,Vietnam - Additional financing for Second Rura...,...,IBRD/IDA,Rural development,Rural services and infrastructure,Energy and Extractives,Climate Change Mitigation and Green House Gase...,P113495,en,0.999996,5117.0,True


In [25]:
complete_metadata_file.head(2)

,corpus,path_original,path_clean,filename_original,year,major_doc_type,doc_type,author,collection,title,...,wb_product_line,wb_major_theme,wb_theme,wb_sector,wb_subtopic_src,wb_project_id,language_detected,language_score,tokens,write_status
id,,,,,,,,,,,,,,,,,,,,,
wb_19200456,wb,/NLP/CORPUS/WB/TXT_ORIG/wb_19200456.txt,/NLP/CORPUS/WB/TXT_CLEAN/wb_19200456.txt,RAD639317460.txt,2014.0,Project Documents,Agreement,NaN,NaN,Official Documents - Amendment to the Suppleme...,...,IBRD/IDA,"Public sector governance,Rural development,Tra...","Public sector governance,Rural services and in...",NaN,"Rural Roads & Transport,Disease Control & Prev...",P094488,en,0.999995,385.0,True
wb_6448332,wb,/NLP/CORPUS/WB/TXT_ORIG/wb_6448332.txt,/NLP/CORPUS/WB/TXT_CLEAN/wb_6448332.txt,E12530EMP1Land0Registry.txt,2005.0,Project Documents,Environmental Assessment,NaN,NaN,Bosnia and Herzegovina - Land Registration Pro...,...,IBRD/IDA,NaN,NaN,"Agriculture, Fishing and Forestry|Financial Se...","Health Care Services Industry,Urban Solid Wast...",P096200,en,0.999996,2451.0,True


In [98]:
complete_metadata_file.to_csv(os.path.join(get_corpus_path(CORPUS_ID), f'{CORPUS_ID.lower()}_pos_metadata_complete.csv'))
complete_metadata_file.head(2)

,corpus,path_original,path_clean,filename_original,year,major_doc_type,doc_type,author,collection,title,...,wb_product_line,wb_major_theme,wb_theme,wb_sector,wb_subtopic_src,wb_project_id,language_detected,language_score,tokens,write_status
id,,,,,,,,,,,,,,,,,,,,,
wb_31189916,wb,/NLP/CORPUS/WB/TXT_ORIG/wb_31189916.txt,/NLP/CORPUS/WB/TXT_CLEAN/wb_31189916.txt,Disclosable-Version-of-the-ISR-Mekong-Delta-In...,2019.0,Project Documents,Implementation Status and Results Report,NaN,NaN,Disclosable Version of the ISR - Mekong Delta ...,...,NaN,NaN,NaN,NaN,"Climate Change Impacts,Labor Markets,Global En...",P153544,en,0.999995,717.0,True
wb_17220866,wb,/NLP/CORPUS/WB/TXT_ORIG/wb_17220866.txt,/NLP/CORPUS/WB/TXT_CLEAN/wb_17220866.txt,E21050v400HA0N0IC0disclosed02050130.txt,2012.0,Project Documents,Environmental Assessment,NaN,NaN,Vietnam - Additional financing for Second Rura...,...,IBRD/IDA,Rural development,Rural services and infrastructure,Energy and Extractives,Climate Change Mitigation and Green House Gase...,P113495,en,0.999996,5117.0,True


In [27]:
# Processing output

In [28]:
s = '''%%time
cleaner = CorpusCleaner(
    use_spellchecker=True, use_respeller=True, use_lemmatizer=True, use_spacy=True,
    replacements_plurals_to_singular_file='whitelist_replacements_plurals_to_singular.csv',
)

cleaner.set_input_folder(TXT_ORIG_DIR)
cleaner.set_output_folder(TXT_CLEAN_DIR)'''

In [29]:
%pprint
print(s)

Pretty printing has been turned OFF
%%time
cleaner = CorpusCleaner(
    use_spellchecker=True, use_respeller=True, use_lemmatizer=True, use_spacy=True,
    replacements_plurals_to_singular_file='whitelist_replacements_plurals_to_singular.csv',
)

cleaner.set_input_folder(TXT_ORIG_DIR)
cleaner.set_output_folder(TXT_CLEAN_DIR)
